In [1]:
from OSLPPWEIGHTED import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import os
import pandas as pd
from torch.optim.lr_scheduler import StepLR
import pickle

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
def get_abbrev(source, target): return source[0].upper() + target[0].upper()
def fmeasures(d): return ' '.join([f'{k}={v*100:.2f}' for (k,v) in d.items()])

In [5]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, i): return self.features[i], self.labels[i]

In [6]:
def _create_balanced_sampler(ds:FeaturesDataset):
    freq2 = Counter(ds.labels.tolist())
    class_weight = {x: 1.0 / freq2[x] for x in freq2}
    source_weights = [class_weight[x] for x in ds.labels.tolist()]
    sampler = torch.utils.data.WeightedRandomSampler(source_weights, len(ds.labels.tolist()))
    return sampler

def _create_trn_dataloader(ds, batch_size, balanced):
    if balanced: return torch.utils.data.DataLoader(ds, batch_size=batch_size, sampler=_create_balanced_sampler(ds), drop_last=True)
    else: torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)

In [7]:
def train_initial_NN(feats_S, lbls_S, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    assert (lbls_S.unique() == torch.arange(num_src_classes)).all()
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats_S, lbls_S)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            
            avg_loss += loss.item()
            optimizer.step()
    return model.eval()

In [8]:
def entropy_loss(logits):
    probs = F.softmax(logits, dim=1)
    entropy =  - probs * probs.log()
    entropy = entropy.sum(dim=1)
    return entropy.mean()

In [9]:
def train_NN(feats, lbls, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    feats, lbls = feats[lbls >= 0], lbls[lbls >= 0]
    feats, lbls = feats[lbls < num_src_classes], lbls[lbls < num_src_classes]
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats, lbls)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            
            avg_loss += loss.item()
            optimizer.step()
    return model.eval()

In [10]:
def predict_NN(model, feats, lbls):
    ds = FeaturesDataset(feats, lbls)
    dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False)
    model = model.eval()
    feats = []
    preds = []
    with torch.no_grad():
        for f,l in dl:
            out = model(f.cuda())
            preds.append(F.softmax(out, dim=1).detach().cpu())
            feats.append(f.detach())
    feats = torch.cat(feats, dim=0)
    preds = torch.cat(preds, dim=0)
    return feats, preds

In [11]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [12]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [13]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [14]:
# sources = ['train'] 
# targets = ['validation']

In [15]:
def set_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [16]:
def reject_by_scores(scores, rejected, selected):
    return (scores < min(scores[rejected == 1].max(), scores[selected == 1].min()))

In [17]:
results = pd.DataFrame({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': [], 'tops': []})
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
#                 for weights in [[0.33, 0.33, 0.33], [0.2, 0.4, 0.4], [0.4, 0.2, 0.4], [0.4, 0.4, 0.2]]:
                 for tops in [[SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY], 
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.MARGIN], 
                                [SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN], 
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN]]:
                    for seed in range(5):
                        set_seed(seed)
                        print(source, '->', target, 'lr=', lr, 'seed=', seed)
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                                  dataset='DomainNet_DCC', source=source, target=target,
                                  num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                        (feats_S, lbls_S), (feats_T, lbls_T) = create_datasets_sub(params.dataset, 
                                                                                   params.source, 
                                                                                   params.target, 
                                                                                   common, 
                                                                                   tgt_private)
                        params.n_r = int(len(lbls_T) * n_r)
                        num_src_classes = params.num_common + params.num_src_priv

                        # l2 normalization and pca
                        feats_S, feats_T = do_l2_normalization(feats_S, feats_T)
                        feats_S, feats_T = do_pca(feats_S, feats_T, params.pca_dim)
                        feats_S, feats_T = do_l2_normalization(feats_S, feats_T)

                        # initial
                        feats_S, feats_T = torch.tensor(feats_S), torch.tensor(feats_T)
                        lbls_S, lbls_T = torch.tensor(lbls_S), torch.tensor(lbls_T)
                        feats_all = torch.cat((feats_S, feats_T), dim=0)
                        pseudo_labels = -torch.ones_like(lbls_T)
                        rejected = torch.zeros_like(pseudo_labels)

                        t = 1

                        model = train_initial_NN(feats_S, lbls_S, epochs, params, balanced=True, lr=lr)
                        feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                        feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                        print((feats_S_2 == feats_S).all(), (feats_T_2 == feats_T).all())
                        assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
#                         selected = select_closed_set_pseudo_labels_weighted(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, weights)
                        selected = select_closed_set_pseudo_labels_tops(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, tops)
                        selected = torch.tensor(selected)
#                         rejected = select_initial_rejected_weighted(confs, params.n_r, preds_T_2, weights)
                        rejected = select_initial_rejected_tops(confs, params.n_r, preds_T_2, tops)

                        rejected = torch.tensor(rejected)
                        selected = selected * (1 - rejected)
            

                        pseudo_labels = cs_pseudo_labels.clone()
                        pseudo_labels[rejected == 1] = num_src_classes
                        pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, 
                                lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                        where = torch.where((selected == 1) + (rejected == 1))[0]
                        metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                        print('______')
                        print(f'Iteration t={t}')
                        print('all: ', fmeasures(metrics))
                        print('selected: ', fmeasures(metrics_selected))

                        for t in range(2, params.T):
                            model = train_NN(feats_all, torch.cat((lbls_S, pseudo_labels), axis=0), epochs, params, balanced=True, lr=lr)
                            feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                            feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                            assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                            confs, cs_pseudo_labels = preds_T_2.max(dim=1)
#                             selected = torch.tensor(select_closed_set_pseudo_labels_weighted(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, weights))
                            selected = torch.tensor(select_closed_set_pseudo_labels_tops(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, tops))
                            selected = selected * (1 - rejected)

#                             scores = get_weighted_scores(confs, preds_T_2, weights)

#                             rejected_new = (scores < min(scores[rejected == 1].max().item(), scores[selected == 1].min().item())).int()

                            conf = get_confidence_scores(confs.numpy()) if SelectRejectMode.CONFIDENCE in tops else None
                            inv_entropy = get_entropy_scores(preds_T_2) if SelectRejectMode.ENTROPY in tops else None
                            margin = get_margin_scores(preds_T_2) if SelectRejectMode.MARGIN in tops else None
            
                            all_rejects = list(filter(lambda x: x is not None, [conf, inv_entropy, margin]))
            
                            mask = np.logical_and.reduce(list(map(lambda x: reject_by_scores(x, rejected, selected), all_rejects)))
                            
                            rejected[mask] = 1
                            selected = selected * (1 - rejected)

                            pseudo_labels = cs_pseudo_labels.clone()
                            pseudo_labels[rejected == 1] = num_src_classes
                            pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                            metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                            where = torch.where((selected == 1) + (rejected == 1))[0]
                            metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                            print('______')
                            print(f'Iteration t={t}')
                            print('all: ', fmeasures(metrics))
                            print('selected: ', fmeasures(metrics_selected))

                        print((pseudo_labels == -1).sum())

                        _rejected = rejected
                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), _rejected.numpy())
                        print('all: ', fmeasures(metrics))

                        results = results.append({'tops': str(tops), 'source': source, 'target': target, 'desc': fmeasures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs}, ignore_index=True)

painting -> sketch lr= 0.001 seed= 0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=51.70 cs/acc_c=53.22 os/recall_knw=90.94 os/recall_unk=28.60 total/acc_i=42.05 total/acc_c=50.27 total/h_score=36.74
selected:  cs/acc_i=59.17 cs/acc_c=60.59 os/recall_knw=76.21 os/recall_unk=75.22 total/acc_i=61.89 total/acc_c=56.88 total/h_score=64.18
______
Iteration t=2
all:  cs/acc_i=50.45 cs/acc_c=52.02 os/recall_knw=86.64 os/recall_unk=36.98 total/acc_i=43.82 total/acc_c=48.82 total/h_score=42.30
selected:  cs/acc_i=54.76 cs/acc_c=56.49 os/recall_knw=79.70 os/recall_unk=58.61 total/acc_i=53.51 total/acc_c=52.84 total/h_score=55.42
______
Iteration t=3
all:  cs/acc_i=51.36 cs/acc_c=52.88 os/recall_knw=86.23 os/recall_unk=38.29 total/acc_i=44.64 total/acc_c=49.31 total/h_score=43.31
selected:  cs/acc_i=53.59 cs/acc_c=55.34 os/recall_knw=83.42 os/recall_unk=47

______
Iteration t=9
all:  cs/acc_i=51.74 cs/acc_c=53.14 os/recall_knw=82.68 os/recall_unk=41.16 total/acc_i=44.76 total/acc_c=47.89 total/h_score=44.42
selected:  cs/acc_i=51.74 cs/acc_c=53.14 os/recall_knw=82.68 os/recall_unk=41.16 total/acc_i=44.76 total/acc_c=47.89 total/h_score=44.42
tensor(0)
all:  cs/acc_i=51.74 cs/acc_c=53.14 os/recall_knw=82.68 os/recall_unk=41.16 total/acc_i=44.76 total/acc_c=47.89 total/h_score=44.42
painting -> sketch lr= 0.001 seed= 3
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=50.30 cs/acc_c=51.54 os/recall_knw=90.19 os/recall_unk=26.77 total/acc_i=40.28 total/acc_c=48.29 total/h_score=34.71
selected:  cs/acc_i=56.17 cs/acc_c=57.58 os/recall_knw=74.73 os/recall_unk=73.56 total/acc_i=58.55 total/acc_c=53.15 total/h_score=61.02
______
Iteration t=2
all:  cs/acc_i=50.68 cs/acc_c=52.14 os/recall_knw=85.58 os/recall_u

______
Iteration t=7
all:  cs/acc_i=52.00 cs/acc_c=53.67 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=46.00 total/acc_c=46.33 total/h_score=47.12
selected:  cs/acc_i=52.00 cs/acc_c=53.67 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=46.00 total/acc_c=46.33 total/h_score=47.12
______
Iteration t=8
all:  cs/acc_i=51.92 cs/acc_c=53.39 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=46.00 total/acc_c=46.33 total/h_score=47.12
selected:  cs/acc_i=51.92 cs/acc_c=53.39 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=46.00 total/acc_c=46.33 total/h_score=47.12
______
Iteration t=9
all:  cs/acc_i=52.53 cs/acc_c=54.07 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=45.98 total/acc_c=46.30 total/h_score=47.10
selected:  cs/acc_i=52.53 cs/acc_c=54.07 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=45.98 total/acc_c=46.30 total/h_score=47.10
tensor(0)
all:  cs/acc_i=52.53 cs/acc_c=54.07 os/recall_knw=76.60 os/recall_unk=48.04 total/acc_i=45.98 total/acc_c=46.30 total/h_

______
Iteration t=5
all:  cs/acc_i=51.02 cs/acc_c=52.58 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.28 total/acc_c=46.70 total/h_score=44.58
selected:  cs/acc_i=51.02 cs/acc_c=52.58 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.28 total/acc_c=46.70 total/h_score=44.58
______
Iteration t=6
all:  cs/acc_i=50.87 cs/acc_c=52.46 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.33 total/acc_c=46.78 total/h_score=44.62
selected:  cs/acc_i=50.87 cs/acc_c=52.46 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.33 total/acc_c=46.78 total/h_score=44.62
______
Iteration t=7
all:  cs/acc_i=51.13 cs/acc_c=52.73 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.33 total/acc_c=46.78 total/h_score=44.62
selected:  cs/acc_i=51.13 cs/acc_c=52.73 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.33 total/acc_c=46.78 total/h_score=44.62
______
Iteration t=8
all:  cs/acc_i=51.21 cs/acc_c=52.83 os/recall_knw=79.81 os/recall_unk=42.47 total/acc_i=44.33 total/acc_c=46.

______
Iteration t=3
all:  cs/acc_i=50.60 cs/acc_c=52.15 os/recall_knw=86.08 os/recall_unk=34.75 total/acc_i=42.44 total/acc_c=47.78 total/h_score=40.46
selected:  cs/acc_i=52.80 cs/acc_c=54.64 os/recall_knw=83.06 os/recall_unk=45.08 total/acc_i=47.14 total/acc_c=49.91 total/h_score=47.48
______
Iteration t=4
all:  cs/acc_i=50.68 cs/acc_c=52.10 os/recall_knw=86.00 os/recall_unk=34.88 total/acc_i=42.75 total/acc_c=48.08 total/h_score=40.66
selected:  cs/acc_i=51.61 cs/acc_c=53.04 os/recall_knw=85.39 os/recall_unk=37.38 total/acc_i=44.23 total/acc_c=48.96 total/h_score=42.61
______
Iteration t=5
all:  cs/acc_i=51.06 cs/acc_c=52.48 os/recall_knw=86.00 os/recall_unk=34.88 total/acc_i=42.70 total/acc_c=47.99 total/h_score=40.63
selected:  cs/acc_i=51.06 cs/acc_c=52.48 os/recall_knw=86.00 os/recall_unk=34.88 total/acc_i=42.70 total/acc_c=47.99 total/h_score=40.63
______
Iteration t=6
all:  cs/acc_i=51.74 cs/acc_c=53.07 os/recall_knw=86.00 os/recall_unk=34.88 total/acc_i=42.84 total/acc_c=48.

...

______
Iteration t=9
all:  cs/acc_i=69.16 cs/acc_c=69.55 os/recall_knw=58.89 os/recall_unk=77.41 total/acc_i=59.90 total/acc_c=53.55 total/h_score=62.47
selected:  cs/acc_i=69.16 cs/acc_c=69.55 os/recall_knw=58.89 os/recall_unk=77.41 total/acc_i=59.90 total/acc_c=53.55 total/h_score=62.47
tensor(0)
all:  cs/acc_i=69.16 cs/acc_c=69.55 os/recall_knw=58.89 os/recall_unk=77.41 total/acc_i=59.90 total/acc_c=53.55 total/h_score=62.47
sketch -> painting lr= 0.001 seed= 2
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=65.20 cs/acc_c=65.88 os/recall_knw=62.23 os/recall_unk=70.35 total/acc_i=58.55 total/acc_c=54.81 total/h_score=61.12
selected:  cs/acc_i=59.17 cs/acc_c=60.73 os/recall_knw=43.86 os/recall_unk=94.43 total/acc_i=59.33 total/acc_c=46.03 total/h_score=59.66
______
Iteration t=2
all:  cs/acc_i=67.01 cs/acc_c=67.51 os/recall_knw=60.31 os/recall_u

______
Iteration t=7
all:  cs/acc_i=68.89 cs/acc_c=69.46 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
selected:  cs/acc_i=68.89 cs/acc_c=69.46 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
______
Iteration t=8
all:  cs/acc_i=68.03 cs/acc_c=68.69 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
selected:  cs/acc_i=68.03 cs/acc_c=68.69 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
______
Iteration t=9
all:  cs/acc_i=68.26 cs/acc_c=68.91 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
selected:  cs/acc_i=68.26 cs/acc_c=68.91 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_score=61.93
tensor(0)
all:  cs/acc_i=68.26 cs/acc_c=68.91 os/recall_knw=60.78 os/recall_unk=73.17 total/acc_i=59.27 total/acc_c=54.61 total/h_

______
Iteration t=5
all:  cs/acc_i=68.69 cs/acc_c=69.35 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.82 total/acc_c=56.81 total/h_score=62.14
selected:  cs/acc_i=68.69 cs/acc_c=69.35 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.82 total/acc_c=56.81 total/h_score=62.14
______
Iteration t=6
all:  cs/acc_i=69.28 cs/acc_c=69.94 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.82 total/acc_c=56.82 total/h_score=62.15
selected:  cs/acc_i=69.28 cs/acc_c=69.94 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.82 total/acc_c=56.82 total/h_score=62.15
______
Iteration t=7
all:  cs/acc_i=68.89 cs/acc_c=69.48 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.80 total/acc_c=56.80 total/h_score=62.13
selected:  cs/acc_i=68.89 cs/acc_c=69.48 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.80 total/acc_c=56.80 total/h_score=62.13
______
Iteration t=8
all:  cs/acc_i=68.81 cs/acc_c=69.50 os/recall_knw=64.93 os/recall_unk=69.52 total/acc_i=59.77 total/acc_c=56.

______
Iteration t=3
all:  cs/acc_i=67.52 cs/acc_c=68.05 os/recall_knw=65.75 os/recall_unk=69.68 total/acc_i=60.12 total/acc_c=57.18 total/h_score=62.43
selected:  cs/acc_i=68.37 cs/acc_c=68.88 os/recall_knw=63.43 os/recall_unk=75.25 total/acc_i=61.88 total/acc_c=57.61 total/h_score=64.69
______
Iteration t=4
all:  cs/acc_i=69.00 cs/acc_c=69.41 os/recall_knw=65.75 os/recall_unk=69.68 total/acc_i=60.25 total/acc_c=57.33 total/h_score=62.53
selected:  cs/acc_i=69.28 cs/acc_c=69.70 os/recall_knw=65.40 os/recall_unk=70.03 total/acc_i=60.47 total/acc_c=57.51 total/h_score=62.78
______
Iteration t=5
all:  cs/acc_i=68.06 cs/acc_c=68.60 os/recall_knw=65.75 os/recall_unk=69.68 total/acc_i=60.25 total/acc_c=57.34 total/h_score=62.54
selected:  cs/acc_i=68.06 cs/acc_c=68.60 os/recall_knw=65.75 os/recall_unk=69.68 total/acc_i=60.25 total/acc_c=57.34 total/h_score=62.54
______
Iteration t=6
all:  cs/acc_i=68.65 cs/acc_c=69.09 os/recall_knw=65.75 os/recall_unk=69.68 total/acc_i=60.25 total/acc_c=57.

tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=64.54 cs/acc_c=65.17 os/recall_knw=67.12 os/recall_unk=66.94 total/acc_i=58.39 total/acc_c=55.85 total/h_score=60.56
selected:  cs/acc_i=59.73 cs/acc_c=61.08 os/recall_knw=46.11 os/recall_unk=94.82 total/acc_i=61.32 total/acc_c=47.64 total/h_score=61.29
______
Iteration t=2
all:  cs/acc_i=67.05 cs/acc_c=67.36 os/recall_knw=66.46 os/recall_unk=68.77 total/acc_i=59.32 total/acc_c=56.26 total/h_score=61.51
selected:  cs/acc_i=68.08 cs/acc_c=68.68 os/recall_knw=59.16 os/recall_unk=84.32 total/acc_i=63.16 total/acc_c=56.07 total/h_score=66.32
______
Iteration t=3
all:  cs/acc_i=67.24 cs/acc_c=67.54 os/recall_knw=66.34 os/recall_unk=69.02 total/acc_i=59.64 total/acc_c=56.59 total/h_score=61.81
selected:  cs/acc_i=68.15 cs/acc_c=68.36 os/recall_knw=64.04 os/recall_unk=73.74 total/acc_i=61.21 total/acc_c=56.92 total/h_score=63.70
______
Iteration t=4
all:  cs/acc_i=68.69 cs/acc_c=69.17 os/recall_knw=66.34 os/recall_unk=69.10 total/a

In [19]:
results.to_csv('results_weighted_scores/dcc__conf__small__NEW__combined_tops__nn_raw.csv', header=True, index=False)

In [20]:
# df = pd.read_csv('office_home_oslpp_nn_raw.csv')
# df['h_score'] = df['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))
# import matplotlib.pyplot as plt
# for n, gr in df.groupby(['source', 'target']):
#     stat = gr.groupby(['lr', 'n_r'])['h_score'].mean()
#     exps = np.array(list(stat.index))
#     fig, ax = plt.subplots()
#     ax.scatter(exps[:, 0], exps[:, 1], c=stat.values)
#     for i, txt in enumerate(stat.values):
#         ax.annotate(round(txt, 2), (exps[i][0], exps[i][1]))
#     plt.title(n)
#     plt.xlabel('lr')
#     plt.ylabel('n_r')
#     plt.xticks(exps[:, 0])
#     plt.yticks(exps[:, 1])
#     fig.colorbar(ax.get_children()[0])
#     plt.show()

In [21]:
results

,source,target,desc,lr,seed,epochs,tops,n_r
0,painting,sketch,cs/acc_i=52.87 cs/acc_c=54.55 os/recall_knw=86...,0.001,0.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10
1,painting,sketch,cs/acc_i=51.40 cs/acc_c=53.16 os/recall_knw=82...,0.001,1.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10
2,painting,sketch,cs/acc_i=51.74 cs/acc_c=53.14 os/recall_knw=82...,0.001,2.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10
3,painting,sketch,cs/acc_i=52.34 cs/acc_c=53.82 os/recall_knw=84...,0.001,3.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10
4,painting,sketch,cs/acc_i=52.38 cs/acc_c=53.95 os/recall_knw=83...,0.001,4.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10
...,...,...,...,...,...,...,...,...
235,sketch,painting,cs/acc_i=68.65 cs/acc_c=69.12 os/recall_knw=65...,0.001,0.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25
236,sketch,painting,cs/acc_i=68.53 cs/acc_c=69.12 os/recall_knw=64...,0.001,1.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25
237,sketch,painting,cs/acc_i=68.46 cs/acc_c=68.99 os/recall_knw=65...,0.001,2.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25
238,sketch,painting,cs/acc_i=68.34 cs/acc_c=68.79 os/recall_knw=66...,0.001,3.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25


In [30]:
results['epoch'] = pd.Series(results.index).apply(lambda i: epochs[(i // 9) % 3])

In [10]:
results = pd.read_csv('results_weighted_scores/dcc__conf__small__NEW__weighted__nn_raw.csv')

In [22]:
wd = results.copy()
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [23]:
wd

,source,target,desc,lr,seed,epochs,tops,n_r,h_score
0,painting,sketch,cs/acc_i=52.87 cs/acc_c=54.55 os/recall_knw=86...,0.001,0.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10,43.83
1,painting,sketch,cs/acc_i=51.40 cs/acc_c=53.16 os/recall_knw=82...,0.001,1.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10,44.81
2,painting,sketch,cs/acc_i=51.74 cs/acc_c=53.14 os/recall_knw=82...,0.001,2.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10,44.42
3,painting,sketch,cs/acc_i=52.34 cs/acc_c=53.82 os/recall_knw=84...,0.001,3.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10,44.86
4,painting,sketch,cs/acc_i=52.38 cs/acc_c=53.95 os/recall_knw=83...,0.001,4.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.10,44.73
...,...,...,...,...,...,...,...,...,...
235,sketch,painting,cs/acc_i=68.65 cs/acc_c=69.12 os/recall_knw=65...,0.001,0.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25,62.59
236,sketch,painting,cs/acc_i=68.53 cs/acc_c=69.12 os/recall_knw=64...,0.001,1.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25,61.80
237,sketch,painting,cs/acc_i=68.46 cs/acc_c=68.99 os/recall_knw=65...,0.001,2.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25,61.96
238,sketch,painting,cs/acc_i=68.34 cs/acc_c=68.79 os/recall_knw=66...,0.001,3.0,10.0,"[<SelectRejectMode.CONFIDENCE: 1>, <SelectReje...",0.25,61.99


In [21]:
wd.groupby(['source', 'target', 'lr', 'n_r', 'epochs', 'weights'])['h_score'].mean().to_frame(name = 'h_score').reset_index().sort_values(by='h_score', ascending=False).drop_duplicates(['source', 'target'])

,source,target,lr,n_r,epochs,weights,h_score
41,sketch,real,0.001,0.1,10.0,"[0.33, 0.33, 0.33]",80.042
2,painting,real,0.001,0.1,10.0,"[0.4, 0.2, 0.4]",78.174
18,real,painting,0.001,0.1,10.0,"[0.4, 0.2, 0.4]",64.244
35,sketch,painting,0.001,0.1,10.0,"[0.4, 0.4, 0.2]",62.856
27,real,sketch,0.001,0.1,10.0,"[0.4, 0.4, 0.2]",55.742
8,painting,sketch,0.001,0.1,10.0,"[0.2, 0.4, 0.4]",49.892


In [26]:
wd['name'] = wd['source'].apply(lambda x: x[0].upper()) + wd['target'].apply(lambda x: x[0].upper())

In [27]:
wd.pivot_table(index=['n_r', 'lr', 'epochs', 'tops'], columns='name', values='h_score', aggfunc=[lambda x: round(np.mean(x), 2)]).sort_index(key=lambda x: x, ascending=False)

<lambda>  \
name                                                                       PR   
n_r  lr    epochs tops                                                          
0.25 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...    77.94   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    77.02   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    77.96   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    78.00   
0.10 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...    76.19   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    76.31   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    77.47   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...    76.17   

                                                                             \
name                                                                     PS   
n_r  lr    epochs tops                                                        
0.25 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  50.74   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  49.71   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  51.36   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  50.87   
0.10 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  42.05   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  45.37   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  44.53   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  42.32   

                                                                             \
name                                                                     RP   
n_r  lr    epochs tops                                                        
0.25 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  66.40   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  65.31   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  66.64   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  66.34   
0.10 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  62.74   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  62.22   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  63.90   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  62.35   

                                                                             \
name                                                                     RS   
n_r  lr    epochs tops                                                        
0.25 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  56.42   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  55.76   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  56.76   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  56.37   
0.10 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  51.45   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  52.81   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  51.73   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  50.94   

                                                                             \
name                                                                     SP   
n_r  lr    epochs tops                                                        
0.25 0.001 10.0   [<SelectRejectMode.ENTROPY: 2>, <SelectRejectMo...  62.18   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  61.90   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  62.53   
                  [<SelectRejectMode.CONFIDENCE: 1>, <SelectRejec...  62.13   
0.10 0.001 10.

In [ ]:
wd.set_index(['lr', 'epoch', 'n_r']).head(30)

In [44]:
wd = pd.read_csv('results/dcc__entropy__StepLR__50_epochs__nn_raw.csv')

In [45]:
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [46]:
wd

,source,target,desc,lr,seed,n_r,h_score
0,painting_train,real_train,cs/acc_i=59.98 cs/acc_c=60.09 os/recall_knw=71...,0.0005,0.0,0.15,54.40
1,painting_train,sketch_train,cs/acc_i=26.40 cs/acc_c=25.41 os/recall_knw=53...,0.0005,0.0,0.15,30.42
2,real_train,painting_train,cs/acc_i=39.74 cs/acc_c=39.39 os/recall_knw=65...,0.0005,0.0,0.15,40.83
3,real_train,sketch_train,cs/acc_i=24.48 cs/acc_c=23.68 os/recall_knw=51...,0.0005,0.0,0.15,29.29
4,sketch_train,painting_train,cs/acc_i=37.51 cs/acc_c=34.45 os/recall_knw=70...,0.0005,0.0,0.15,35.06
5,sketch_train,real_train,cs/acc_i=56.05 cs/acc_c=57.00 os/recall_knw=71...,0.0005,0.0,0.15,48.56
